In [ ]:
import mlflow
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

from hyperopt import fmin, tpe, Trials, STATUS_OK, hp, space_eval
from hyperopt.pyll import scope
from functools import partial

from pickle import dump

from scripts.Preprocessing import Preprocessing
from scripts.BinaryClassificationTraining import BinaryClassificationTraining

from scripts.config import (year_month_train, 
    input_data_path_train,
    seed)

In [ ]:
local_path_save = './local_artifacts_tmp/07_GB_XGB/'
year_month = year_month_train
input_data_path = input_data_path_train

### MLFlow setting

In [ ]:
if not os.path.exists(local_path_save):
    os.makedirs(local_path_save)

#save all metadata in a sqlite db. Artifacts will be saved on local folder ./mlflow    
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Name of the experiment
exp_name = "07 - GB and XGB"
# set up MlFlow axperiment
experiment_id = mlflow.set_experiment(exp_name)

## Gradient Boosting (sklearn implementation)

Write here your experiments, using what we have already done in the previous sections

In [ ]:
#TODO

## XGB with categorical variables

In [ ]:
run_name = 'xgb_categorical'

In [ ]:
xgb_cat_training = BinaryClassificationTraining(
    input_data_path,
    local_path_save,
    year_month,
    'xgb_cat')

In [ ]:
prepr = Preprocessing(input_data_path_train, task_type='classification')
X, Y = prepr.read_dataframe(request_tgt=True)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=seed)


#preprocessing ohe
shapes_pre = (X_train.shape[0], X_test.shape[0])
X_train_cat, _, scaler = prepr.preprocess_for_classification(df=X_train, 
                                                               enable_categorical=True, 
                                                               perform_scaling=True)
X_test_cat, _, _ = prepr.preprocess_for_classification(df=X_test, 
                                                    enable_categorical=True,
                                                    perform_scaling=True,
                                                    scaler=scaler)
assert shapes_pre == (X_train.shape[0], X_test.shape[0])
dump(scaler, open(local_path_save + run_name + '_scaler.pkl', 'wb'))

In [ ]:
# Calculating the baseline

xgb_cat_training.calculate_classification_baseline(Y_train=Y_train, 
                                               Y_test=Y_test, run_name=run_name)


In [ ]:
max_evals = 3

# Here we can decide which hyperparameters we want to tune
xgboost_cat_parameters_search = {
    # https://xgboost-clone.readthedocs.io/en/latest/parameter.html#learning-task-parameters
    "objective":'binary:logistic',
    "tree_method": "hist",
    #"n_estimators": scope.int(hp.quniform('n_estimators', 5, 20, 1)),
    "n_estimators": 100,
    "eval_metric": "auc", 
    "max_depth": hp.choice('max_depth', np.arange(1, 5, dtype=int)), #xgb alllows only int max_depth (max_depth=1.0 will raise an error)
    'eta': hp.uniform('eta', 0.5, 2),
    #"colsample_bylevel": 0.7,# stocastic gb (at each split(level) randomly choose between 70% of features )
    'enable_categorical': True, #if categorical data are present (i.e. fetures with pandas type='Category') they are managed, If they are not present and 'enable_categorical=True --> no problem
    #"early_stopping_rounds": 10,
    "seed": seed
}

xgb_cat_training.set_hyperparameter_space(xgboost_cat_parameters_search)

In [ ]:
trials = Trials()

best_result = fmin(
    fn=partial(xgb_cat_training.objective_xgb, 
        X_train=X_train_cat,
        X_test=X_test_cat,
        Y_train=Y_train,
        Y_test=Y_test,
        save_ohe=False,
        run_name=run_name,
        threshold=0.5),
    space=xgb_cat_training.hp_space,
    algo=tpe.suggest,
    max_evals=max_evals,
    trials=trials,
    rstate=np.random.default_rng(seed)
)

best_result

## XGB with OHE

In [ ]:
run_name = 'xgb_ohe'

In [ ]:
xgb_ohe_training = BinaryClassificationTraining(
    input_data_path,
    local_path_save,
    year_month,
    'xgb_ohe')

In [ ]:
prepr = Preprocessing(input_data_path_train, task_type='classification')
X, Y = prepr.read_dataframe(request_tgt=True)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=seed)


#preprocessing ohe
shapes_pre = (X_train.shape[0], X_test.shape[0])
X_train_ohe, ohe, scaler = prepr.preprocess_for_classification(df=X_train, 
                                                               fit_ohe=True,
                                                               perform_scaling=True)
X_test_ohe, _, _ = prepr.preprocess_for_classification(df=X_test, 
                                                    ohe=ohe,
                                                    perform_scaling=True,
                                                    scaler=scaler)
assert shapes_pre == (X_train.shape[0], X_test.shape[0])
dump(ohe, open(local_path_save + run_name + '_ohe.pkl', 'wb'))
dump(scaler, open(local_path_save + run_name + '_scaler.pkl', 'wb'))

In [ ]:
# Calculating the baseline

xgb_ohe_training.calculate_classification_baseline(Y_train=Y_train, 
                                               Y_test=Y_test, run_name=run_name)


In [ ]:
max_evals = 3

# Here we can decide which hyperparameters we want to tune
xgboost_ohe_parameters_search = {
    # https://xgboost-clone.readthedocs.io/en/latest/parameter.html#learning-task-parameters
    "objective":'binary:logistic',
    "tree_method": "hist",
    #"n_estimators": scope.int(hp.quniform('n_estimators', 5, 20, 1)),
    "n_estimators": 100,
    "eval_metric": "auc", 
    "max_depth": hp.choice('max_depth', np.arange(1, 5, dtype=int)), #xgb alllows only int max_depth (max_depth=1.0 will raise an error)
    'eta': hp.uniform('eta', 0.5, 2),
    #"colsample_bylevel": 0.7,# stocastic gb (at each split(level) randomly choose between 70% of features )
    'enable_categorical': False, #if categorical data are present (i.e. fetures with pandas type='Category') they are managed, If they are not present and 'enable_categorical=True --> no problem
    #"early_stopping_rounds": 10,
    "seed": seed
}

xgb_ohe_training.set_hyperparameter_space(xgboost_ohe_parameters_search)

In [ ]:
trials = Trials()

best_result = fmin(
    fn=partial(xgb_ohe_training.objective_xgb, 
        X_train=X_train_ohe,
        X_test=X_test_ohe,
        Y_train=Y_train,
        Y_test=Y_test,
        run_name=run_name,
        threshold=0.5),
    space=xgb_ohe_training.hp_space,
    algo=tpe.suggest,
    max_evals=max_evals,
    trials=trials,
    rstate=np.random.default_rng(seed)
)

best_result

### Learning Curve of the best model

In [ ]:
from sklearn.model_selection import learning_curve
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, make_scorer

In [ ]:
roc_auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

In [ ]:
best_hps = xgboost_ohe_parameters_search.copy()
best_hps['eta'] = best_result['eta']
best_hps['max_depth'] = best_result['max_depth']

In [ ]:
best_hps

In [ ]:
best_xgb_ohe = XGBClassifier(**best_hps)

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(estimator=best_xgb_ohe, X=X_train_ohe, y=Y_train,
                                                       cv=10, train_sizes=np.linspace(0.1, 1.0, 10),
                                                     n_jobs=-1,
                                                       scoring=roc_auc_scorer)

In [ ]:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training ROC AUC')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation ROC AUC')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Model ROC AUC')
plt.grid()
plt.legend(loc='lower right')
plt.show()